In [1]:
import sys
sys.path.append("../../")

In [9]:
from scipy.signal import find_peaks
import librosa
import os.path
import pickle as pkl
import json

sys.path.append("./dependencies/hps")
sys.path.append("./dependencies/hps/preprocess")

from xsens import mvnx
from global_vars import *
from unity_functions import process_smpl_pose, process_smpl_trans
from velocity_filter import get_cam


def get_allpaths(dir_name, extension):
    """
    Obtain all the paths with the extension in the directory provided
    :param dir_name: The name of the directory wh
    :param extension:
    :return:
    """
    return_list = []
    for dirpath, dirnames, filenames in os.walk(dir_name):
        for filename in [f for f in filenames if f.endswith(extension)]:
            return_list.append(os.path.join(dirpath, filename))
    return return_list

def get_contacts(contacts):
    """
    Reformarts mvnx contacts
    :param contacts:
    :return: Contacts for all the four parts of a foot in the required dictionary format
    """
    LeftFoot_Toe = []
    RightFoot_Toe = []

    LeftFoot_Heel = []
    RightFoot_Heel = []

    for i in range(len(contacts)):
        LeftFoot_Heel.append(contacts[i]['LeftFoot_Heel'][0])
        RightFoot_Heel.append(contacts[i]['RightFoot_Heel'][0])

        LeftFoot_Toe.append(contacts[i]['LeftFoot_Toe'][0])
        RightFoot_Toe.append(contacts[i]['RightFoot_Toe'][0])

    new_dict = {
        'left_heel':LeftFoot_Heel,
        'left_toe': LeftFoot_Toe,
        'right_toe':RightFoot_Toe,
        'right_heel':RightFoot_Heel
    }

    return new_dict

def get_cam_start(video_name, audio_name):
    """
    Find the starting point of the camera by detecting the first peak in the audio file
    :param video_name: The name of the video file
    :param audio_name: The name of the audio file associated with the video
    :return: The frame at which the clap is detected in the camera input
    """

    if not os.path.exists(audio_name):
        clip = AudioFileClip(video_name)
        clip.write_audiofile(audio_name)

    audio, sr = librosa.load(audio_name, sr = None)
    if len(audio) > 0:
        audio_short =audio[:50*sr]
        print(audio_short)
        peaks, dict_ret = find_peaks(audio_short, height = 0)

        num = np.where(audio_short == np.max(dict_ret['peak_heights']))
        cam_start = float(num[0]*30)/sr
        return int(cam_start)
    else:
        return int(0)

def get_imu_start(acc):
    """
    Find the start of the IMU stream by detecting the acceleration peak
    :param acc: input acceleration
    :return: The frame at which the clap is detected in the imu stream
    """
    rt_acc = []
    lt_acc = []
    for i in range(len(acc)):
        rt_acc.append(acc[i]['right_hand'])
        lt_acc.append(acc[i]['left_hand'])

    rt_acc = np.linalg.norm(np.array(rt_acc[:1000]), axis =1)
    lt_acc = np.linalg.norm(np.array(lt_acc[:1000]), axis = 1)

    peaks_rt, dict_ret_rt = find_peaks(rt_acc, height = 0)
    peaks_lt, dict_ret_lt = find_peaks(lt_acc, height = 0)

    rt_max = np.where(rt_acc == np.max(dict_ret_rt['peak_heights']))
    lt_max = np.where(lt_acc == np.max(dict_ret_lt['peak_heights']))

    print('Right start: ', rt_max[0][0])
    print('Left Start: ', lt_max[0][0])
    return int(lt_max[0][0])

def get_video_name(filename):
    """
    Get the video file name from the sequence name
    :param filename: The sequence name
    :return:
    """
    all_videos = get_allpaths(VIDEO_PATH, ".mp4")
    for name in all_videos:
        if (filename + ".mp4" in name):
            return name

def get_cam_file_name(filename):
    """
    Get the camera localization results file name from the sequence name
    :param filename:
    :return: The name of the camera localization result
    """
    all_jsons = get_allpaths(IN_CAM_PATH, ".json")
    for name in all_jsons:
        if (filename + ".json" in name) :
            return name

In [10]:
# mvnx_name = "./SUB2_MPI_EG-001.mvnx" # read error 
mvnx_name = "./SUB5_MPI_GEB-002.mvnx"

data = mvnx.MVNX(mvnx_name)

In [11]:
len(data.frames)
data.available_info

['acceleration',
 'angularAcceleration',
 'angularVelocity',
 'centerOfMass',
 'footContacts',
 'jointAngle',
 'jointAngleErgo',
 'jointAngleErgoXZY',
 'jointAngleXZY',
 'orientation',
 'position',
 'sensorFreeAcceleration',
 'sensorMagneticField',
 'sensorOrientation',
 'velocity']

In [13]:
# cam_start = get_cam_start(video_name, audio_name)

acc = data.get_info("acceleration")
imu_start = get_imu_start(acc)

init_dict = {
    'imu_start' : imu_start,
#     'cam_start' : cam_start
}

print('IMU starts at : {}'.format(imu_start))
# print('Camera starts at : {}'.format(cam_start))

Right start:  41
Left Start:  42
IMU starts at : 42
